<a href="https://colab.research.google.com/github/sidharth-ds/Keras---Deep-Learning/blob/main/keras_iris_multiclass_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [5]:
# load dataset
dataframe = pd.read_csv("iris.csv", header=None)
dataframe

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [6]:
# separate features and target variables
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

## Encode the Output Variable:
* When modeling multi-class classification problems using neural networks, it is good practice to reshape the output attribute using one hot encoding or creating dummy variables from a categorical variable.
* In this problem three class values are Iris-setosa, Iris-versicolor and Iris-virginica

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

## Define & Compile the Neural Network Model:

In [8]:
# define baseline model:

def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

* The function creates a baseline neural network for the iris classification problem. It creates a simple fully connected network with one hidden layer that contains 8 neurons.
  * 4 inputs -> [8 hidden nodes] -> 3 outputs
* The hidden layer uses a **rectifier activation** function.
* Because we used a one-hot encoding for our iris dataset, the output layer must create 3 output values, one for each class. 
  * The output value with the **largest value** will be taken as the class predicted by the model.
* We use a **softmax activation** function in the output layer.
    * This is to ensure the output values are in the **range of 0 and 1** and may be used as predicted probabilities.
* The network uses the efficient **Adam gradient descent** optimization algorithm with a **logarithmic loss** function, which is called **categorical_crossentropy** in Keras.

## create KerasClassifier for use in scikit-learn:

In [9]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


* Debugging is turned off when training by setting verbose to 0

## Evaluate The Model with k-Fold Cross Validation:
* The gold standard for evaluating machine learning models is k-fold cross validation.

In [10]:
kfold = KFold(n_splits=10, shuffle=True)

In [11]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 98.00% (3.06%)


* **result** - describes the evaluation of the 10 constructed models for each of the splits of the dataset.
* The results are summarized as both the **mean (98%)** and **standard deviation (3.06%)** of the model accuracy on the dataset